## Modeller 10.4
Updated on 07/16/2025



## 00. Installation

In [10]:
%%bash
# -------------------------------
# Install Dependencies & Modeller (bash cell)
# -------------------------------

# 1. Install Python dependencies
pip install biopython pdb-tools py3Dmol

# 2. Download & unpack Modeller 10.4
wget -q https://salilab.org/modeller/10.4/modeller-10.4.tar.gz
tar -zxf modeller-10.4.tar.gz

# 3. Configure installer with your license key
cat > modeller_config <<EOF
2
$(pwd)/compiled/MODELLER
MODELIRANJE
EOF

# 4. Run installer and symlink mod10.4
mkdir -p compiled/MODELLER
cd modeller-10.4
./Install < ../modeller_config
ln -sf $(pwd)/../compiled/MODELLER/bin/mod10.4 /usr/local/bin/mod10.4

# 5. Verify installation
mod10.4 | head -1   # should print "usage:" if installed correctly


Installation of MODELLER 10.4

This script will install MODELLER 10.4 into a specified directory
for which you have read/write permissions.

To accept the default answers indicated in [...], press <Enter> only.

------------------------------------------------------------------------

The currently supported architectures are as follows:

   1) Linux x86 PC (e.g. RedHat, SuSe).
   2) x86_64 (Opteron/EM64T) box (Linux).
   3) Alternative x86 Linux binary (e.g. for FreeBSD).
   4) Linux on 32-bit ARM (e.g. for Raspberry Pi).
   5) Linux on 64-bit ARM (e.g. for Raspberry Pi 3 or later).

Select the type of your computer from the list above [2]: 

Full directory name in which to install MODELLER 10.4
[/root/bin/modeller10.4]: 

License key, obtained from our academic license server at
https://salilab.org/modeller/registration.html: 
------------------------------------------------------------------------

The following settings will be used: 

The type of this machine          : x86_64-int

## 01 Modeling the protein

In [ ]:
%%bash
# -------------------------------
# Full Pipeline Fix: Remodeling with Modeller & CRYST1 Preservation
# -------------------------------

# --- Customize these ---
pdb_id="1lyz"
uniprot_id="P00698"
region_len=148

# -------------------------------
# 1) Fetch input files
# -------------------------------
wget -q https://files.rcsb.org/download/${pdb_id^^}.pdb
wget -q https://www.uniprot.org/uniprot/${uniprot_id}.fasta -O ${uniprot_id}.fasta
mkdir -p work && mv ${pdb_id^^}.pdb ${uniprot_id}.fasta work/
cd work
cp ${pdb_id^^}.pdb ${pdb_id}_orig.pdb

# -------------------------------
# 2) Clean PDB & extract CRYST1
# -------------------------------
python3 << 'PYCODE'
from Bio.PDB import PDBParser, PDBIO, Select
pdb="1lyz"
# grab CRYST1
cryst1 = next((l for l in open(f"{pdb}_orig.pdb") if l.startswith("CRYST1")), "")
# strip hetero/missing
class Std(Select):
    def accept_residue(self, r): return r.id[0]==' '
parser = PDBParser(QUIET=True)
structure = parser.get_structure("S", f"{pdb}_orig.pdb")
io = PDBIO(); io.set_structure(structure)
io.save(f"{pdb}_clean.pdb", Std())
with open("cryst1.txt","w") as f: f.write(cryst1)
print("✅ Cleaned PDB and extracted CRYST1")
PYCODE

# -------------------------------
# 3) Reconcile SEQRES vs UniProt
# -------------------------------
python3 << 'PYCODE'
from Bio import SeqIO
from Bio.Data.IUPACData import protein_letters_3to1 as aa3to1
pdb, uni, L = "1lyz", "P00698", 148
# SEQRES → one-letter
seqres = ""
for l in open(f"{pdb}_orig.pdb"):
    if l.startswith("SEQRES") and l[11]=="A":
        seqres += "".join(aa3to1.get(r.capitalize(),"X") for r in l.split()[4:])
uni_seq = str(SeqIO.read(f"{uni}.fasta","fasta").seq)[:L]
if seqres != uni_seq:
    print("⚠️ Mismatch; using UniProt slice.")
    seqres = uni_seq
else:
    print("✅ SEQRES matches UniProt.")
with open("final_seq.txt","w") as f: f.write(seqres)
PYCODE

# -------------------------------
# 4) Build template sequence
# -------------------------------
python3 << 'PYCODE'
from Bio.PDB import PDBParser
from Bio.Data.IUPACData import protein_letters_3to1 as aa3to1
pdb, L = "1lyz", 148
parser = PDBParser(QUIET=True)
s = parser.get_structure("S", f"{pdb}_clean.pdb")
chain = next(s[0].get_chains())
obs = {r.id[1]: aa3to1.get(r.get_resname().capitalize(),"X")
       for r in chain if r.id[0]==' '}
template = "".join(obs.get(i, "-") for i in range(1, L+1))
with open("template_seq.txt","w") as f: f.write(template)
print("✅ Template sequence built")
PYCODE

# -------------------------------
# 5) Write PIR alignment
# -------------------------------
python3 << 'PYCODE'
pdb, L = "1lyz", 148
templ = open("template_seq.txt").read().strip()
target = open("final_seq.txt").read().strip()
start = templ.find(next(c for c in templ if c!="-")) + 1
end   = templ.rfind(next(c for c in templ[::-1] if c!="-")) + 1
with open("alignment.ali","w") as f:
    f.write(f">P1;{pdb}\n")
    f.write(f"structureX:{pdb}:{start}:A:{end}:A::::\n{templ}*\n")
    f.write(">P1;target\n")
    f.write(f"sequence:target:1:A:{L}:A::::\n{target}*\n")
print("✅ alignment.ali written")
PYCODE

# -------------------------------
# 6) Run Modeller
# -------------------------------
mod10.4 << 'EOF'
from modeller import *
from modeller.automodel import *
env = environ()
env.io.hetatm = True
a = automodel(env,
              alnfile='alignment.ali',
              knowns='1lyz',
              sequence='target',
              assess_methods=(assess.DOPE, assess.GA341))
a.starting_model = 1
a.ending_model   = 1
a.make()
EOF

# -------------------------------
# 7) Re-insert CRYST1 into models
# -------------------------------
python3 << 'PYCODE'
import glob
cryst = open("cryst1.txt").read()
pdb = "1lyz"
for fn in glob.glob("target*.pdb") + glob.glob(f"{pdb}_clean.pdb"):
    out = fn.replace(".pdb", "_with_cryst.pdb")
    with open(fn) as inp, open(out, "w") as o:
        inserted = False
        for l in inp:
            if (l.startswith("ATOM") or l.startswith("HETATM")) and not inserted:
                o.write(cryst)
                inserted = True
            if not l.startswith("CRYST1"):
                o.write(l)
        if not inserted:
            o.write(cryst)
    print(f"✔ {out}")
print("✅ All models have CRYST1 inserted.")
PYCODE


✅ Cleaned PDB and extracted CRYST1
⚠️ Mismatch; using UniProt slice.
✅ Template sequence built
✅ alignment.ali written
usage: mod10.4 script [...]

✔ 1lyz_clean_with_cryst.pdb
✅ All models have CRYST1 inserted.


# 02 Visualize the protein

---



In [11]:
# -------------------------------
# Step 9: Visualize Model and Sequence (Colorful + Numbering)
# -------------------------------
import py3Dmol
from Bio.PDB import PDBParser

# Select first modeled PDB
model_file = glob.glob('*_with_cryst.pdb')[0]

# Extract and format sequence with numbering
parser = PDBParser(QUIET=True)
structure = parser.get_structure('model', model_file)
chain_A = structure[0]['A']
seq = ''.join(three_to_one(res.get_resname()) for res in chain_A if res.id[0]==' ')
chunk_size = 60
for i in range(0, len(seq), chunk_size):
    segment = seq[i:i+chunk_size]
    # Build ruler with numbers every 10 residues
    ruler = [' ']*len(segment)
    for j in range(len(segment)):
        if (j+1) % 10 == 0:
            label = str(i + j + 1)
            start_pos = j - len(label) + 1
            ruler[start_pos:j+1] = list(label)
    print(''.join(ruler))
    print(segment)

# 3D Visualization with spectrum coloring by residue index
view = py3Dmol.view(width=800, height=500)
with open(model_file) as f:
    pdb_data = f.read()
view.addModel(pdb_data, 'pdb')
# Apply spectrum coloring: rainbow from N- to C-terminus
view.setStyle({'chain':'A'}, {'cartoon': {'color':'spectrum'}})
view.setBackgroundColor('0xFFFFFF')
view.zoomTo({'chain':'A'})
view.show()


NameError: name 'glob' is not defined